<a href="https://colab.research.google.com/github/marvinxu-free/Algorithm/blob/master/tensorflow2_chapter2_%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E6%95%B0%E5%AD%A6%E5%9F%BA%E7%A1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 神经网络数学基础

主要为了加深神经网络基础数据的了解



## 初识神经网络
神经网络的基本组件是layer，可以把它当成数据过滤器：

- 进入的是一些数据，出来的数据是更加有效的数据

要想训练网络，还需要编译三个关键参数：

1. 损失函数（loss function): 如何衡量网络在训练数据上的性能。
2. 优化器（optimizer）：基于训练数据和损失函数来更新网络。
3. 在训练和测试过程中需要监控的指标（metric): 比如准确率，auc，recall等。


In [19]:
import tensorflow as tf
import numpy as np
mnist = tf.keras.datasets.mnist
(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x, test_x = train_x/255.0, test_x/255.0
epoch = 10

## keras function API

keras定义模型有两种方式：
- 使用Sequential类（仅用于基于层的线性堆叠，这是目前最常用的网络架构），但是结构比较简单，代码可读性较差
- function API模式，用于层组成的有向无环图，可以构建任意形式的架构.
- function API也有缺点：不支持动态架构，这在大部分的神经网络中都是不需要的，但是对于递归的神经网络或者tree RNN中必须要有动态架构。

因此，一般如果网络不需要支持recurse编程的话，使用函数式编程会比较好，如果需要支持的话，那么就需要混合使用各种编程模式。

比如简单的mnist数据集上的分类模型：

### subclass模式（一般不建议使用）

subclass模式值得是模型代码通过形式`tf.keras.Model`的构建模型，使用这种方式构建模型，有个特点，就是在模型类中必须手工重载实现其 call() 方法。换句话来说，就是 call() 方法的参数 training 必须由你来管理了。怎么理解呢，下面举个例子：

```python
def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()   
        self.dense1 = tf.keras.layers.Dense(units=100)
        self.bn = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(units=10)

def call(self, inputs, training=None):         
    x = self.flatten(inputs)    
    x = tf.nn.relu(self.bn(self.dense1(x), training=training))  
    output = tf.nn.softmax(self.dense2(x))
    return output
```

在上面这个例子中，BatchNormalization 层的实例 self.bn 在调用过程中，需要我们手工传入 training 参数，使得它在训练模式和推理模式下表现出不同的行为。

而在前面两种构建模型的方式中，我们不需要手工管理 training 参数，只需要在使用模型类实例 model 时，隐式调用其 call() 方法，传入正确的 training 参数值即可，TensorFlow 在底层构建模型时就会根据我们传入的 training 参数值，自动给 BatchNormalization 层等赋予不同的行为。

所以，在前面两种构建模型的方式中，我们不需要在 BatchNormalization 层的实例被调用时传入 training 参数值，而第三种方式则需要！

此外subclass模式出来的模型不能直接调用summary()函数查看网络架构。


In [30]:
class MyModel(tf.keras.Model):

  def __init__(self, num_class=10):
    super(MyModel, self).__init__()
    # 定义layer
    # inputs = tf.keras.Input(shape=(28,28)) # 返回一个placeholder用于数据输入
    self.x0 = tf.keras.layers.Flatten()
    self.x1 = tf.keras.layers.Dense(512, activation='relu', name='d1')
    self.x2 = tf.keras.layers.Dropout(0.2)
    self.predictions = tf.keras.layers.Dense(10, activation='softmax', name='d2')

  def call(self, inputs):
    """
    function API定义前向传递计算逻辑的范式，
    基于的是Python的科利华编程技巧

    """
    # 使用前面`__init__`定义的神经网络层
    x = self.x0(inputs)
    x = self.x1(x)
    x = self.x2(x)
    return self.predictions(x)



In [35]:
model = MyModel()

In [32]:
optimiser = tf.keras.optimizers.Adam()
model.compile (optimizer= optimiser, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=epoch)

Epoch 1/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2185 - accuracy: 0.9347
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0964 - accuracy: 0.9704
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0694 - accuracy: 0.9786
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0529 - accuracy: 0.9829
Epoch 5/10
1875/1875 [==============================] - 8s 5ms/step - loss: 0.0431 - accuracy: 0.9858
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0368 - accuracy: 0.9876
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0320 - accuracy: 0.9894
Epoch 8/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0269 - accuracy: 0.9907
Epoch 9/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0248 - accuracy: 0.9918
Epoch 10/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.022

### function API

比较pure的function API

In [25]:
# use keras functional API
inputs = tf.keras.Input(shape=(28,28))  # Returns a placeholder tensor
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(512, activation='relu',name='d1')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(10,activation=tf.nn.softmax, name='d2')(x)

In [26]:
model3 = tf.keras.Model(inputs=inputs, outputs=predictions)

In [27]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
d1 (Dense)                   (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
d2 (Dense)                   (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [28]:
optimiser = tf.keras.optimizers.Adam()
model3.compile (optimizer= optimiser, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [29]:
model3.fit(train_x, train_y, batch_size=32, epochs=epoch)

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2208 - accuracy: 0.9342
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0956 - accuracy: 0.9699
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0688 - accuracy: 0.9781
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0519 - accuracy: 0.9839
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0421 - accuracy: 0.9864
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0359 - accuracy: 0.9882
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0316 - accuracy: 0.9896
Epoch 8/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0272 - accuracy: 0.9911
Epoch 9/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0242 - accuracy: 0.9924
Epoch 10/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.022

## 神经网络数学表示

### 张量（tensor)
张量是一个数据容器，其包含的数据几乎都是数值数据，因此其是一个数字容器。比如矩阵，其就是一个二维张量。

张量是矩阵向任意维度的推广，==**注意张量的维度通常叫做axis**==

### 标量（0D张量）

仅包含一个数字的张量，叫做标量，也叫作零维张量、0D张量。在numpy中，一个数字就是一个标量，可以通过np.ndim查看一个标量的轴的个数。比如:


In [36]:
import numpy as np
x = np.array(1)
x.ndim

0

### 向量（1D张量）

数字组成的数组叫做向量(vector)或者一维张量（1D张量），一维张量只有一个轴。比如:

In [37]:
a = np.arange(10)
a.ndim

1

### dimension容易混淆的点
上面的a有10个元素，在很多概念里面dimension既能表示沿某个轴的元素的个数（比如5D向量),也可以表示向量中轴的个数（比如5D张量），有时候会让人感到混乱。
对于后一种情况，严格的说应该是5阶张量（张量的阶数代表的就是轴的个数），但是5D张量的说法更加常见。

### 矩阵（2D张量）
向量组成的数组称作矩阵（matrix）或者2D张量。矩阵有两个轴，通常第一和称作行，第二个称作列。。

In [38]:
x = np.arange(100).reshape([10,10])
x.ndim

2

### 3D张量或者更高维度的张量
将多个矩阵组合成数组，就变成3D张量。
将多个3D张量组合成数组，就变成了4D张量。
将多个4D张量组合成数组，就变成了5D张量。

深度学习处理的数据一般是0-4D的张量，但处理视频数据的时候回用到5D张量.

### 张量的关键属性
张量有以下几个关键属性:
1. 轴的个数（阶）：例如3D张量有3个轴，4D张量有4个轴。
2. 形状：这是一个整数元祖，代表每一个轴上面的维度大小(元素的个数)，例如前面的矩阵的形状就是(10,10).
3. 数据类型：张量中所包含的数据类型，一般不包括字符串类型，因为张量存储的是预先分配的连续内存段，而字符串是变长的，无法用这种方式存储。

### 张量切片操作
选择张量的特定元素叫做张量的切片操作。

In [40]:
x = np.arange(100).reshape([10,10])
print(x.shape)
x[1:10, 7].shape

(10, 10)


(9,)

### 数据批量的概念
通常来说，在深度学习中第一个轴都是样本轴（sample axis，0轴，因为索引是从0开始）。在MNIST的例子中，样本就是图像。

此外，在深度学习中，模型不会同时处理整个数据集，而是将数据分为小批量，具体就是MNIST数据集的一个批量，批量大小为128.
`batch = train_image[:128]`
然后是下一个批量:
`batch = train_image[128:256]`

**对于这种批量张量，第一个轴（0轴）叫做批量轴**。

### 现实中的数据张量

遇到的数据示例几乎是以下之一：

- 向量数据： 2D张量，形状为(samples, features)：比如人口统计数据，每个人表示为包含年龄、学历的向量，10000个人就是(10000,3)的2D张量。
- 时间序列或者序列数据: 3D张量(samples, timestamp, features)
- 图像: 4D张量(samples, height, width, channel)
- 视频: 5D张量(samples,frame, height, width, channel)或者(samples,frame, channel, height, width)

### 张量运算
神经网络学到额所有变化，都可以简化为数值数据张量上的一些张量运算(tensor operation)。例如加上张量、乘以张量。


#### 逐元素运算（element-wise)

element-wise运算，即张量中的所有元素可以独立计算，非常适合大规模的并行计算。

比如矩阵加法: `x + x`

逐元素运算，只支持两个形状相同的张量。

#### 广播运算
如果两个形状不同的张量相加,在没有歧义的情况下将小的张量进行广播，广播包含以下两个步骤：
- 向较小的张量添加轴， 是ndim与大的相同
- 将较小的张量沿着新轴复制，使得两者大小相同。
- 然后进行逐元素计算。

实际中并不会真的复制，只是在算法里面计算逻辑有体现。

#### 点积运算
点积运算，也叫作张量积（Tensor product， 不要和element-wise计算混淆),**是最有用的也是最重要的张量运算**。与逐元素计算不同，它将输入的元素合并到一起。

有以下几个点：
- 向量的点击是标量。
- 如果一个张量的ndim大于1，点积不满足交换律。
- 对于矩阵x和y， 当且仅当x.shape[1]==y.shape[0]的时候，才能进行点积dot(x,y)
- 更一般的说，对更高的维度进行点积，其形状必须遵循前面的2D向量相同的原则.

```python
(a,b,c,d) . (d) -> (a, b, c)
(a,b,c,d) . (d,e) -> (a,b,c,e)
```
#### 张量变形
要求size相等。


### 张量运算的几何解释
可以将神经网络解释为高维空间非常复杂的几何变换。